# OpenCV - Procesamiento de imagenes

## Manipulación de imagenes

In [14]:
import cv2

img = cv2.imread("sample6.jpg", cv2.IMREAD_COLOR)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print(img.shape)
print(gray.shape)

cv2.imshow("Imagen BGR", img)
cv2.imshow("Imagen Gray", gray)

cv2.waitKey(0)
cv2.destroyAllWindows()

(450, 600, 3)
(450, 600)


## Region of Image (ROI)

In [15]:
import cv2

img = cv2.imread("sample6.jpg", cv2.IMREAD_COLOR)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img_roi = img[100:350,250:400,:] 
gray_roi = gray[100:350,250:400]

cv2.imshow("Imagen BGR", img_roi)
cv2.imshow("Imagen Gray", gray_roi)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Dibujo de figuras con OpenCV

In [17]:
import cv2

img = cv2.imread("sample2.jpg", cv2.IMREAD_COLOR)
cv2.line(img, (50, 100), (50, 400), (0, 255, 0), 2)   # imagen, punto de inicio, punto fianl, color, grosor
                                                                # (columna de pixel,fila de pixel)
cv2.imshow("Imagen", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
import cv2

img = cv2.imread("sample6.jpg", cv2.IMREAD_COLOR)
cv2.rectangle(img, (250, 120), (400, 300), (0, 255, 0), 2)  

cv2.imshow("Imagen", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
import cv2

img = cv2.imread("sample3.jpg", cv2.IMREAD_COLOR)
cv2.circle(img, (250, 400), 100, (0, 255, 0), 2)   # imagen, centro, radio, color, grosor

cv2.imshow("Imagen", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Image Blending (addWeighted)
Se puede realizar una suma de imagenes, asignando un peso porcentual a cada imagen. Esto requiere que ambas imagenes tengan las mismas dimensiones y los mismos canales de colores (es decir, la misma forma). Los parametros de la formula (alpha, beta y gama) estan definidos para:

$$dst = \alpha * img1 + \beta * imag + \gamma $$

In [19]:
import cv2

img1 = cv2.imread("sample2.jpg", cv2.IMREAD_COLOR)
print(img1.shape)

img2 = cv2.imread("sample1.jpg", cv2.IMREAD_COLOR)
img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
print(img2.shape)

img_photo = cv2.addWeighted(img1, 0.7, img2, 0.3, 0) 
cv2.imshow("Imagen", img_photo)

cv2.waitKey(0)
cv2.destroyAllWindows()

(500, 667, 3)
(500, 667, 3)


In [20]:
import cv2

cap = cv2.VideoCapture(0)
img = cv2.imread("monalisa.jpg")


while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape
    img = cv2.resize(img, (width, height))
    
    img_blended = cv2.addWeighted(img, 0.7, frame, 0.3, 0)
    cv2.imshow('MonaLisa', img_blended)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


## Operaciones lógicas (bitwise)
Se puede realizar operaciones lógicas con los arreglos de imagenes a novel de OpenCV, asi como operaciones aritmeticas. Con estas operaciones se pueden hacer tratamientos especiales a las imagenes.

In [22]:
import cv2

img1 = cv2.imread("sample4.jpg", cv2.IMREAD_COLOR)
img2 = cv2.imread("upc_logo.jpg", cv2.IMREAD_COLOR)
img2 = cv2.resize(img2, (80, 80))
roi = img1[-80:, -80:]

cv2.imshow("img1", img1)
cv2.imshow("img2", img2)

# Se crea una mascara con la imagen del logo con un control de umbral (threshold)
# y se invierte la imagen con una operacion logica (bitwise_not)
img_gray= cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img_gray, 128, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

cv2.imshow('mask', mask)
cv2.imshow('mask_inv', mask_inv)

# Hay que oscurecer los pixels que seran reemplazados por el logo (ROI)
img_bg = cv2.bitwise_and(roi, roi, mask=mask)
cv2.imshow("img1_bg", img_bg)

# Hay que tomar solo los puntos que conforman en logo (mascara)
img_fg = cv2.bitwise_and(img2, img2, mask=mask_inv)
cv2.imshow("img2_fg", img_fg)

# Hay que sumar ambas imagenes en el ROI
dst = cv2.add(img_bg, img_fg)
img1[-80:, -80:] = dst
cv2.imshow("Imagen con logo", img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
import cv2

def put_logo(img, logo, roi):
    gray= cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    fondo = cv2.bitwise_and(roi, roi, mask=mask)
    frente = cv2.bitwise_and(logo, logo, mask=mask_inv)

    dst = cv2.add(fondo, frente)
    img[-100:-20, -80:] = dst
    
    return img

cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    logo = cv2.imread("upc_logo.jpg", cv2.IMREAD_COLOR)
    logo = cv2.resize(logo, (80, 80))
    roi = img[-100:-20, -80:]
    
    img_with_logo = put_logo(img, logo, roi)
    
    cv2.imshow("Video Logo", img_with_logo)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


 ## Mascaras con inRange (Deteccion de colores)
 ![](https://i.stack.imgur.com/gyuw4.png)

In [25]:
import cv2
import numpy as np

img = cv2.imread("people_green_bg.jpg", cv2.IMREAD_COLOR)
img_bg = cv2.imread("office_bg.jpg", cv2.IMREAD_COLOR)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

print(img_hsv[10, 10, :])    # HUE, SATURATION, VALUE

lower_bound = np.array([40, 200, 255])
upper_bound = np.array([85, 255, 255])
mask = cv2.inRange(img_hsv, lower_bound, upper_bound)
mask_inv = cv2.bitwise_not(mask)

kernel = np.ones((3, 3), np.uint8)
mask = cv2.dilate(mask, kernel, iterations=1)
mask_inv = cv2.erode(mask_inv, kernel, iterations=1)

img_mask = cv2.bitwise_and(img, img, mask=mask_inv)

cv2.imshow('mask', mask)
cv2.imshow('img', img)
cv2.imshow('img_mask', img_mask)

roi = img_bg[-img_mask.shape[0]:,-img_mask.shape[1]:,:]
img_bg2 = cv2.bitwise_and(roi, roi, mask=mask)

dst = cv2.add(img_mask, img_bg2)
cv2.imshow('Final', dst)

cv2.waitKey(0)
cv2.destroyAllWindows()

[ 57 238 255]


## Ejercicio
Modifique el kiosko de fotos para que incluya una imagen donde pueda superponer su rostro (como quien mete la cabeza por un agujero en un fondo).

In [51]:
# PHOTO: face_here.jpg
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
ret, frame = cap.read()

img = cv2.imread("face_here.jpg")
img = cv2.resize(img, (frame.shape[1], frame.shape[0]))
roi_img = img[58:195, 270:355]

lower_bound = np.array([200, 200, 200])
upper_bound = np.array([255, 255, 255])
mask = cv2.inRange(roi_img, lower_bound, upper_bound)
mask_inv = cv2.bitwise_not(mask)

while True:
    ret, frame = cap.read()
    
    roi_frame = frame[58:195, 270:355]
    res1 = cv2.bitwise_and(roi_img, roi_img, mask=mask_inv)
    res2 = cv2.bitwise_and(roi_frame, roi_frame, mask=mask)
    dst = cv2.add(res1, res2)

    img[58:195, 270:355] = dst
    
    cv2.imshow('WebCam Color', img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

## Filtrado de imagenes

In [50]:
import cv2
import numpy as np

img = cv2.imread("sample5.jpg")
img = cv2.resize(img, (300, 300))
cv2.imshow("Imagen", img)

kernel = np.ones((5, 5), np.float32) / 25

img_fil1 = cv2.filter2D(img, -1, kernel)
img_fil2 = cv2.blur(img, (5, 5))
img_fil3 = cv2.GaussianBlur(img, (7, 7), 0)

cv2.imshow('Filter2D', img_fil1)
cv2.imshow('Blur', img_fil2)
cv2.imshow('Gaussian Filter', img_fil3)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [49]:
import cv2
import numpy as np

img = cv2.imread("noise_image.jpg")
img = cv2.resize(img, (400, 300))
cv2.imshow("Imagen", img)

img_fil1 = cv2.medianBlur(img, 5)     #contorno borroso
img_fil2 = cv2.bilateralFilter(img, 9, 75, 75)

cv2.imshow("Median Blur", img_fil1)
cv2.imshow("Bilateral Filter", img_fil2)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Detección de bordes

In [46]:
import cv2

img = cv2.imread("sample2.jpg", cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (400, 300))
cv2.imshow("Imagen", img)

edges = cv2.Canny(img, 100, 200)
cv2.imshow('edges', edges)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    edges = cv2.Canny(frame, 100, 200)
    edges_inv = cv2.bitwise_not(edges)
    
    cv2.imshow('edges', edges_inv)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# Detección de contornos

In [39]:
import cv2
import numpy as np

# Se detectan objetos de color rojo, verde o azul
cap = cv2.VideoCapture(0)

kernel_open = np.ones((30, 30))
kernel_close = np.ones((100, 100))

while True:
    ret, frame = cap.read()
    
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    mask1 = cv2.inRange(img_hsv, (0,50,20), (5,255,255))   # Rango inferior rojo
    mask2 = cv2.inRange(img_hsv, (175,50,20), (180,255,255))    # Rango superior rojo
    mask = cv2.bitwise_or(mask1, mask2)
    
    # erosion + dilation: elimina puntos blancos
    mask_open = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_open)          
    # dilation + erosion: elimina agujeros negros
    mask_close = cv2.morphologyEx(mask_open, cv2.MORPH_CLOSE, kernel_close)
    
    mask_final = mask_close
    
    # Se hallan los contornos de la imagen (blanco sobre fondo negro)
    conts, h = cv2.findContours(mask_final, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.drawContours(frame, conts, -1, (255, 0, 0), 3)
    
    for i in range(len(conts)):
        x, y, w, h = cv2.boundingRect(conts[i])
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
           
    cv2.imshow("WebCam", frame)
    cv2.imshow("Mask Final", mask_final)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
    

### Haar Cascade Detection

In [36]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
smile_cascade = cv2.CascadeClassifier("haarcascade_smile.xml")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)
    
    for x, y, w, h in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        smiles = smile_cascade.detectMultiScale(roi_gray, 1.8, 20)
        
        for sx, sy, sw, sh in smiles:
            cv2.rectangle(roi_color, (sx, sy), (sx+sw, sy+sh), (0, 0, 255), 2)
       
    
    cv2.imshow('WebCam', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

## Ejercicio
Automatice el kiosko de fotos en tkinter para que pueda tomar fotografias de forma automática cuando todas las personas frente a la camara se encuentren mirando a la cámara y sonriendo.